In [435]:
import tensorflow as tf
import numpy as np
import time
import csv
import matplotlib.pyplot as plt

np.set_printoptions(precision=4, suppress=True)

In [436]:
seed=0

mnist = tf.keras.datasets.mnist
(x_train_data, y_train_cold),(x_test_data,y_test_cold) = mnist.load_data()
x_train_data, x_test_data = x_train_data / 255.0, x_test_data / 255.0

In [437]:
with tf.Session() as sess:
    y_train = sess.run(tf.one_hot(y_train_cold,10))
    y_test =  sess.run(tf.one_hot(y_test_cold,10))

In [438]:
x_train=[]
x_test=[]

n_train = len(x_train_data)
n_test = len(x_test_data)

for i in range(len(x_train_data)):
    x_train.append(np.ndarray.flatten(x_train_data[i]))
    
for i in range(len(x_test_data)):
    x_test.append(np.ndarray.flatten(x_test_data[i]))

In [439]:
x_train = np.reshape(x_train,[n_train,784])
x_test = np.reshape(x_test,[n_test, 784])

In [440]:
sigma=0.001
batch_size=1000

In [441]:
def build_graph(n_hl, update_rule, num_hidden_layers):
    #later can include passing non-linearity in the function as well!
    #include dataset independence while creating the graph & separate function for loading data!
    #different function to create all the variables & operations!
    g=tf.Graph()
    with g.as_default():        
        # init=tf.global_variables_initializer()
        
        X = tf.placeholder(dtype=tf.float32, shape=[None, 784], name='x')
        Y = tf.placeholder(dtype=tf.float32, shape=[None,10], name='y')
        
        xavier = tf.contrib.layers.xavier_initializer(seed=0)
        
        w={}
        b={}
        h={}
        x={}
        s={}
        h_star={}
        x_star={}
        del_h={}
        delta_w={}
        delta_b={}
        
        update_w={}
        update_b={}
        
        s['in']=tf.zeros([1], dtype=tf.float32, name='s_in')
        for i in range(num_hidden_layers+1):
            s[str(i)]=tf.zeros([1], dtype=tf.float32, name='s_'+str(i))
            
        for i in range(num_hidden_layers+1):
            if(i==0):
                w[str(i)]=tf.Variable(xavier([n_hl, 784]))
                b[str(i)]=tf.Variable(xavier([n_hl]))
                h[str(i)]=tf.matmul(X, tf.transpose(w[str(i)]))
                x[str(i)]=tf.nn.relu(tf.add(h[str(i)], b[str(i)]))
                if(update_rule=='np' or update_rule=='np-hybrid'):
                    s[str(i)]=tf.random_normal(shape = [batch_size,n_hl], mean=0, stddev=sigma, dtype=tf.float32, name='s_'+str(i), seed=seed)
                if(update_rule=='ip'):
                    s['in']=tf.random_normal(shape = [batch_size,784], mean=0, stddev=sigma, dtype=tf.float32, name='s_in', seed=seed)
                    
            elif (i<num_hidden_layers):
                w[str(i)]=tf.Variable(xavier([n_hl, n_hl]))
                b[str(i)]=tf.Variable(xavier([n_hl]))
                h[str(i)]=tf.matmul(x[str(i-1)], tf.transpose(w[str(i)]))
                x[str(i)]=tf.nn.relu(tf.add(h[str(i)], b[str(i)]))
                if(update_rule=='np' or update_rule=='np-hybrid'):
                    s[str(i)]=tf.random_normal(shape = [batch_size,n_hl], mean=0, stddev=sigma, dtype=tf.float32, name='s_'+str(i), seed=seed)
            else:
                w[str(i)]=tf.Variable(xavier([10, n_hl]))
                b[str(i)]=tf.Variable(xavier([10]))
                h[str(i)]=tf.matmul(x[str(i-1)], tf.transpose(w[str(i)]))
                pred=tf.nn.softmax(tf.add(h[str(i)], b[str(i)]))
                if(update_rule=='np' or update_rule=='np-hybrid'):
                    s[str(i)]=tf.random_normal(shape = [batch_size, 10], mean=0, stddev=sigma, dtype=tf.float32, name='s_'+str(i), seed=seed)
                               
        lr = tf.Variable(xavier([1]) , dtype=tf.float32, name='lr')
        
        error=tf.reduce_mean(np.square(pred-Y),1)
        mse=tf.losses.mean_squared_error(predictions=pred, labels=Y)
        
        if(update_rule=='ip' or update_rule=='np' or update_rule=='np-hybrid'):    
            for i in range(num_hidden_layers+1):
                if(i==0):                               
                    h_star[str(i)]=tf.matmul(tf.add(X,s['in']), tf.transpose(w[str(i)]))    
                else:
                    h_star[str(i)]=tf.matmul(x_star[str(i-1)], tf.transpose(w[str(i)]))    
                
                if(i<num_hidden_layers):                
                    x_star[str(i)]=tf.nn.relu(tf.add(tf.add(h_star[str(i)], b[str(i)]), s[str(i)]))
                    
                else:
                    pred_star=tf.nn.softmax(tf.add(tf.add(h_star[str(i)], b[str(i)]), s[str(i)]))
            
            error_star=tf.reduce_mean(np.square(pred_star-Y),1)
            var = sigma ** 2
            k = -lr * (error_star - error) / var
        
            for i in range(num_hidden_layers+1):
                if(update_rule=='ip'):
                    del_h[str(i)]=h_star[str(i)]-h[str(i)]
    
                elif(update_rule=='np'):
                    del_h[str(i)]=s[str(i)]
                
                elif(update_rule=='np-hybrid'):
                    del_h[str(i)]=s[str(i)]
                    k=k*(1-alpha)
                    
                    
                if(i==0):
                    delta_w[str(i)]=tf.einsum('ki,kj->kij', del_h[str(i)], X)
                
                else:
                    delta_w[str(i)]=tf.einsum('ki,kj->kij', del_h[str(i)], x[str(i-1)])
                    
                delta_w[str(i)]=tf.einsum('kij,k->kij',delta_w[str(i)],k)
                delta_w[str(i)]=tf.reduce_mean(delta_w[str(i)],0)
                delta_b[str(i)]=tf.einsum('ki,k->ki',del_h[str(i)],k)
                delta_b[str(i)]=tf.reduce_mean(delta_b[str(i)],0)
                
                update_w[str(i)]=tf.assign(w[str(i)], tf.add(w[str(i)],delta_w[str(i)]), name='update_w'+str(i))         
                update_b[str(i)]=tf.assign(b[str(i)], tf.add(b[str(i)],delta_b[str(i)]), name='update_b'+str(i))
         
        change_lr=tf.assign(lr, tf.reshape(learning_rate, [1]), name='change_lr')
                
        if(update_rule=='np-hybrid'):
            optimizer=tf.train.GradientDescentOptimizer(learning_rate=alpha*lr[0], name='optimizer').minimize(mse)
        elif(update_rule=='sgd'):
            optimizer=tf.train.GradientDescentOptimizer(learning_rate=lr[0], name='optimizer').minimize(mse)
        # train_step=optimizer.minimize(mse, name='train_step')

        is_correct = tf.equal(tf.argmax(pred,1), tf.argmax(Y,1))
        accuracy = 100*tf.reduce_mean(tf.cast(is_correct, tf.float32))
        tf.identity(accuracy, 'accuracy')
    
    return g

In [442]:
def find_acc(which_acc):
    acc=[]
    if(which_acc=='test'):
        acc.append(sess.run('accuracy:0', feed_dict={'x:0':x_test, 'y:0':y_test}))

    elif(which_acc=='train'):
        n=int(n_train/n_test)
        for i in range(n):
            acc.append(sess.run('accuracy:0', feed_dict={'x:0':x_train[i*n_test:(i+1)*n_test], 'y:0':y_train[i*n_test:(i+1)*n_test]}))
    else:
        print('wrong accuracy requested!!')
    return np.mean(acc)

In [443]:
def print_acc():    
    train_acc.append(find_acc('train'))
    test_acc.append(find_acc('test'))
    print('epoch : ', i+1, '  test_acc : ', test_acc[-1])
    sec=int(time.time()-start)
    print(int(sec/60),'m ', int(sec%60),'s')
    
    return test_acc[-1] 

In [444]:
def train_network(num_hidden_layers):
    for j in range(int(n_train/batch_size)):
        ind = np.random.randint(0, n_train, size=(batch_size))
        
        if(update_rule=='sgd' or update_rule=='np-hybrid'):
            ops=['optimizer']
            sess.run(ops, feed_dict = {'x:0':x_train[ind], 'y:0':y_train[ind]})
            
        if(update_rule=='ip' or update_rule=='np' or update_rule=='np-hybrid'):
            updates=[]
            squiggles=[]
            for i in range(num_hidden_layers+1):
                updates.append('update_w'+str(i)+':0')
                updates.append('update_b'+str(i)+':0')
                squiggles.append('s_'+str(i)+':0')
            
            ops=[squiggles, updates]
            sess.run(ops, feed_dict = {'x:0':x_train[ind], 'y:0':y_train[ind]})

    return

In [445]:
def write_in_file():
    row=test_acc
    with open('hybrid-test.csv', 'a') as csvFile:
        writer = csv.writer(csvFile)
        writer.writerow(row)
    
    csvFile.close()
    return

In [446]:
n_epochs=5
interval=1

hl_size = 300
learning_rate = 0.1
num_hidden_layers=1
test_acc=[]
train_acc=[]
update_rule='np-hybrid'
alpha=0

In [447]:
start=time.time()

tf.set_random_seed(seed)
np.random.seed(seed)
graph=build_graph(hl_size, update_rule, num_hidden_layers)
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    sess.run('change_lr:0')
    print('TRAINING : ', hl_size,' hidden units', '\nlearning rate : ', sess.run('lr:0'), '\n')
    for i in range(n_epochs):
        if i%interval==0:
            if(print_acc()>98):
                break
            if i>10 and find_acc('test')<15:
                break    
                
        train_network(num_hidden_layers)
    
    sess.close()
    # write_in_file()

sec=int(time.time()-start)
print('TOTAL TIME : ', int(sec/60),'m ', int(sec%60),'s')

TRAINING :  300  hidden units 
learning rate :  [0.1] 

epoch :  1   test_acc :  9.56
0 m  0 s
epoch :  2   test_acc :  17.720001
0 m  2 s
epoch :  3   test_acc :  38.280003
0 m  4 s
epoch :  4   test_acc :  52.569996
0 m  5 s
epoch :  5   test_acc :  58.56
0 m  7 s
TOTAL TIME :  0 m  9 s


In [448]:
# fig, ax1 = plt.subplots(figsize=(8, 5.5))
# xx=np.arange(0,n_epochs, interval)
# ax1.set_xlabel('epochs')
# ax1.set_ylabel('accuracy')
# ax1.plot(xx, test_acc, color='tab:red', label='test accuracy')
# ax1.plot(xx, train_acc, color='tab:blue', label='training accuracy')
# ax1.legend()
# 
# axes = plt.gca()
# 
# ax1.set_facecolor("#ffffb3")
# 
# ax1.annotate(str(round(test_acc[-1],1)),xy=(xx[-1]-0.5,test_acc[-1]+0.7), color='tab:red')
# ax1.annotate(str(round(train_acc[-1],1)),xy=(xx[-1]-0.5,train_acc[-1]-2.2), color='tab:blue')
# 
# plt.grid(True, color="#93a1a1", alpha=0.3)
# plt.show()
# plt.savefig('plot1.svg', dpi=500)